In [79]:
import pandas as pd

In [80]:
import numpy as np

In [81]:
jogos = pd.read_csv("data/jogos-resultado.csv")

In [82]:
del jogos["Unnamed: 0"]

In [83]:
jogos.head(3)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,brasileiro-a,2018,6,21-05,Internacional,Chapecoense,3,0,MANDANTE
1,brasileiro-a,2018,6,20-05,Fluminense,Atlético-PR,2,0,MANDANTE
2,brasileiro-a,2018,6,20-05,América-MG,Botafogo,1,0,MANDANTE


### Selecionar temporadas

In [84]:
temporadas = list(jogos["ano"].unique())

In [85]:
def classificar(jogos, rodada, classificacao):
    
    classificacao_nova = classificacao.copy()
    
    for jogo in jogos[jogos["rodada"] == rodada].iterrows():
        jogo_info = jogo[1];
        if (jogo_info["resultado"] == "MANDANTE"):            
            classificacao_nova.loc[jogo_info["mandante"]]["pts"] = classificacao.loc[jogo_info["mandante"]]["pts"] + 3
            classificacao_nova.loc[jogo_info["mandante"]]["v"] = classificacao.loc[jogo_info["mandante"]]["v"] + 1            
        elif (jogo_info["resultado"] == "VISITANTE"):
            classificacao_nova.loc[jogo_info["visitante"]]["pts"] = classificacao.loc[jogo_info["visitante"]]["pts"] + 3
            classificacao_nova.loc[jogo_info["visitante"]]["v"] = classificacao.loc[jogo_info["visitante"]]["v"] + 1
        elif (jogo_info["resultado"] == "EMPATE"):     
            classificacao_nova.loc[jogo_info["mandante"]]["pts"] = classificacao.loc[jogo_info["mandante"]]["pts"] + 1
            classificacao_nova.loc[jogo_info["visitante"]]["pts"] = classificacao.loc[jogo_info["visitante"]]["pts"] + 1
            
        classificacao_nova.loc[jogo_info["mandante"]]["gp"] = classificacao.loc[jogo_info["mandante"]]["gp"] + jogo_info["gols_mandante"]    
        classificacao_nova.loc[jogo_info["mandante"]]["gc"] = classificacao.loc[jogo_info["mandante"]]["gc"] + jogo_info["gols_visitante"]
        
        classificacao_nova.loc[jogo_info["visitante"]]["gp"] = classificacao.loc[jogo_info["visitante"]]["gp"] + jogo_info["gols_visitante"]    
        classificacao_nova.loc[jogo_info["visitante"]]["gc"] = classificacao.loc[jogo_info["visitante"]]["gc"] + jogo_info["gols_mandante"]    
        
        classificacao_nova.loc[jogo_info["mandante"]]["jogos"] = classificacao.loc[jogo_info["mandante"]]["jogos"] + 1    
        classificacao_nova.loc[jogo_info["visitante"]]["jogos"] = classificacao.loc[jogo_info["visitante"]]["jogos"] + 1
    
    classificacao_nova.eval("sg = gp - gc", inplace=True)
    classificacao_nova.sort_values(["pts", "v", "sg", "gp"], ascending=[0,0,0,0], inplace=True)
        
    for i in range(1, len(classificacao_nova.index.values.tolist())+1):   
        classificacao_nova.loc[classificacao_nova.index[i-1]]["posicao"] = i
    
    return classificacao_nova

In [86]:
classificacao_DF = {}

In [87]:
for temporada in temporadas:         
    
    rodadas = list(jogos.query("ano == " + str(temporada)).sort_values("rodada")["rodada"].unique())    
    times = list(jogos.query("ano == " + str(temporada))["mandante"].unique())
 
    classificacao_DF[temporada] = {}
    
    classificacao = pd.DataFrame({"time":times, "posicao": np.zeros(len(times)), "jogos": np.zeros(len(times)), "pts": np.zeros(len(times)), "v":np.zeros(len(times)), "gp":np.zeros(len(times)), "gc":np.zeros(len(times)), "sg":np.zeros(len(times))})
    classificacao.set_index('time', inplace=True)
    
    for rodada in rodadas: 
        if rodada == 1:
            classificacao_DF[temporada][rodada] = classificar(jogos.query("ano == " + str(temporada) + " and rodada == " + str(rodada)), rodada, classificacao)
        else:
            classificacao_DF[temporada][rodada] = classificar(jogos.query("ano == " + str(temporada) + " and rodada == " + str(rodada)), rodada, classificacao_DF[temporada][rodada-1])

In [88]:
def getPosicao(classificacao, row, tipo):
    return classificacao[row["ano"]][row["rodada"]].loc[row[tipo]]["posicao"]

In [89]:
jogos["posicao_m"] = jogos.apply(lambda row: getPosicao(classificacao_DF, row, "mandante"),axis=1)

In [90]:
jogos["posicao_v"] = jogos.apply(lambda row: getPosicao(classificacao_DF, row, "visitante"),axis=1)

In [91]:
jogos.head(10)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v
0,brasileiro-a,2018,6,21-05,Internacional,Chapecoense,3,0,MANDANTE,10.0,16.0
1,brasileiro-a,2018,6,20-05,Fluminense,Atlético-PR,2,0,MANDANTE,5.0,17.0
2,brasileiro-a,2018,6,20-05,América-MG,Botafogo,1,0,MANDANTE,6.0,11.0
3,brasileiro-a,2018,6,20-05,Paraná,Grêmio,0,0,EMPATE,20.0,8.0
4,brasileiro-a,2018,6,20-05,São Paulo,Santos,1,0,MANDANTE,7.0,15.0
5,brasileiro-a,2018,6,20-05,Sport Recife,Corinthians,1,1,EMPATE,12.0,3.0
6,brasileiro-a,2018,6,20-05,Vitória,Ceará,2,1,MANDANTE,14.0,19.0
7,brasileiro-a,2018,6,19-05,Palmeiras,Bahia,3,0,MANDANTE,4.0,18.0
8,brasileiro-a,2018,6,19-05,Flamengo,Vasco,1,1,EMPATE,2.0,9.0
9,brasileiro-a,2018,6,19-05,Atlético-MG,Cruzeiro,1,0,MANDANTE,1.0,13.0


In [92]:
classificacao_DF[2018][6]

,gc,gp,jogos,posicao,pts,sg,v
time,,,,,,,
Atlético-MG,6.0,9.0,6.0,1.0,13.0,3.0,4.0
Flamengo,6.0,12.0,6.0,2.0,11.0,6.0,3.0
Corinthians,4.0,9.0,6.0,3.0,11.0,5.0,3.0
Palmeiras,3.0,8.0,6.0,4.0,11.0,5.0,3.0
Fluminense,6.0,8.0,6.0,5.0,10.0,2.0,3.0
América-MG,9.0,9.0,6.0,6.0,10.0,0.0,3.0
São Paulo,5.0,7.0,6.0,7.0,10.0,2.0,2.0
Grêmio,3.0,7.0,6.0,8.0,9.0,4.0,2.0
Vasco,7.0,10.0,5.0,9.0,8.0,3.0,2.0


In [93]:
def aproveitamento5Rodadas (classificacao, row, tipo, jogos):    
    
    rodada = row["rodada"] - 1    
    rodada5r = rodada - jogos
    
    if rodada < 1:
        rodada = 1
        
    if rodada5r < 1:
        rodada5r = 1    
        
    pontos = classificacao[row["ano"]][rodada].loc[row[tipo]]["pts"]
    pontos5r = classificacao[row["ano"]][rodada5r].loc[row[tipo]]["pts"]
    
    if rodada == rodada5r:
        return 0        
    
    return round((pontos - pontos5r)/((rodada - rodada5r)*3), 2)

In [94]:
jogos["aproveitamento_5r_v"] = jogos.apply(lambda row: aproveitamento5Rodadas(classificacao_DF, row, "visitante", 5),axis=1)

In [95]:
jogos["aproveitamento_5r_m"] = jogos.apply(lambda row: aproveitamento5Rodadas(classificacao_DF, row, "mandante", 5),axis=1)

In [96]:
jogos["aproveitamento_10r_v"] = jogos.apply(lambda row: aproveitamento5Rodadas(classificacao_DF, row, "visitante", 10),axis=1)

In [97]:
jogos["aproveitamento_10r_m"] = jogos.apply(lambda row: aproveitamento5Rodadas(classificacao_DF, row, "mandante", 10),axis=1)

In [98]:
jogos.head(10)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
0,brasileiro-a,2018,6,21-05,Internacional,Chapecoense,3,0,MANDANTE,10.0,16.0,0.50,0.17,0.50,0.17
1,brasileiro-a,2018,6,20-05,Fluminense,Atlético-PR,2,0,MANDANTE,5.0,17.0,0.17,0.58,0.17,0.58
2,brasileiro-a,2018,6,20-05,América-MG,Botafogo,1,0,MANDANTE,6.0,11.0,0.58,0.33,0.58,0.33
3,brasileiro-a,2018,6,20-05,Paraná,Grêmio,0,0,EMPATE,20.0,8.0,0.42,0.08,0.42,0.08
4,brasileiro-a,2018,6,20-05,São Paulo,Santos,1,0,MANDANTE,7.0,15.0,0.25,0.33,0.25,0.33
5,brasileiro-a,2018,6,20-05,Sport Recife,Corinthians,1,1,EMPATE,12.0,3.0,0.58,0.58,0.58,0.58
6,brasileiro-a,2018,6,20-05,Vitória,Ceará,2,1,MANDANTE,14.0,19.0,0.25,0.25,0.25,0.25
7,brasileiro-a,2018,6,19-05,Palmeiras,Bahia,3,0,MANDANTE,4.0,18.0,0.42,0.58,0.42,0.58
8,brasileiro-a,2018,6,19-05,Flamengo,Vasco,1,1,EMPATE,2.0,9.0,0.33,0.75,0.33,0.75
9,brasileiro-a,2018,6,19-05,Atlético-MG,Cruzeiro,1,0,MANDANTE,1.0,13.0,0.58,0.83,0.58,0.83


In [99]:
jogos.to_csv(path_or_buf="data/jogos-resultado-posicao.csv",sep=",")